# Create ML & DL algo

In [254]:
import numpy as np
import pandas as pd
import tensorflow as tf 

In [255]:
# data loading from .npz
npz = np.load('seawater_data_train.npz')
train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.float)

npz = np.load('seawater_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.float)

npz = np.load('seawater_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.float)


C:\Users\parmarn\AppData\Local\Temp\ipykernel_364\794442931.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_inputs = npz['inputs'].astype(np.float)
C:\Users\parmarn\AppData\Local\Temp\ipykernel_364\794442931.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_targets = npz['targets'].astype(np.float)
C:\Users\parmarn\AppDa

## ANN Model

Outline, Optimizers, loss function, early stoping and training

In [256]:
# model prep
input_size = 6
output_size = 1
hidden_layer_size = 10

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='linear')
])


In [257]:
# model prep, optimizer and loss function
model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError(),metrics=[tf.keras.metrics.MeanAbsoluteError()])
              

In [258]:
# Training
BATCH_SIZE = 1
MAX_EPOCHS = 50
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

In [259]:
# fit the model
model.fit(
    train_inputs, train_targets, batch_size=BATCH_SIZE,epochs=MAX_EPOCHS, callbacks=[early_stopping], validation_data=(validation_inputs, validation_targets), verbose=2
          )

Epoch 1/50
17/17 - 1s - loss: 34.3514 - mean_absolute_error: 5.4857 - val_loss: 19.6086 - val_mean_absolute_error: 4.3441 - 1s/epoch - 88ms/step
Epoch 2/50
17/17 - 0s - loss: 33.2476 - mean_absolute_error: 5.3881 - val_loss: 18.3434 - val_mean_absolute_error: 4.2084 - 73ms/epoch - 4ms/step
Epoch 3/50
17/17 - 0s - loss: 32.1755 - mean_absolute_error: 5.2992 - val_loss: 17.0403 - val_mean_absolute_error: 4.0627 - 79ms/epoch - 5ms/step
Epoch 4/50
17/17 - 0s - loss: 31.1124 - mean_absolute_error: 5.2085 - val_loss: 15.7157 - val_mean_absolute_error: 3.9073 - 75ms/epoch - 4ms/step
Epoch 5/50
17/17 - 0s - loss: 29.8600 - mean_absolute_error: 5.1047 - val_loss: 14.6363 - val_mean_absolute_error: 3.7733 - 81ms/epoch - 5ms/step
Epoch 6/50
17/17 - 0s - loss: 28.7125 - mean_absolute_error: 5.0010 - val_loss: 13.6228 - val_mean_absolute_error: 3.6398 - 90ms/epoch - 5ms/step
Epoch 7/50
17/17 - 0s - loss: 27.6248 - mean_absolute_error: 4.8971 - val_loss: 12.6009 - val_mean_absolute_error: 3.4990 - 7

# Model Test

In [260]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

1/1 [==============================] - 0s 197ms/step - loss: 2.7474 - mean_absolute_error: 1.2738


In [261]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 2.75. Test accuracy: 127.38%


In [262]:
prediction = model.predict(test_inputs)
prediction = pd.DataFrame(prediction)
test_targets = pd.DataFrame(test_targets)

result = pd.concat([test_targets, prediction], axis = 1, ignore_index=True)
result.columns = ['Targets', 'prediction']

result

1/1 [==============================] - 0s 85ms/step


,Targets,prediction
0,5.54,2.768682
1,3.61,3.158350
2,5.44,4.841622


# Model Deployment

In [263]:
# deploying trained model
import keras
model.save('sw_model.h5')